In [1]:
import eunomia
from eunomia.agents import Eunomia
from eunomia.tools import EunomiaTools

In [2]:
import logging
logging.getLogger().setLevel(logging.CRITICAL)
import warnings

# Disable all warning messages
warnings.filterwarnings("ignore")

## Obtaining Water Stability Information from a MOF Paper

In [3]:
paper_id = 69
paper_file =f'../Data/chosen_papers/{paper_id}.pdf'
docs_processor = eunomia.LoadDoc(file_name=paper_file, encoding="utf8")
sliced_pages = docs_processor.process(['references ', 'acknowledgement', 'acknowledgments', 'references\n'],
                                             chunk_size=1000, chunk_overlap=20, chunking_type='fixed-size')

In [4]:
from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings

Embedding_model = 'text-embedding-ada-002' 
faiss_index_path = f"../Data/chosen_papers/faiss/{Embedding_model}/faiss_index_{Embedding_model}_{paper_id}.pkl"
faiss_index = FAISS.from_documents(sliced_pages, OpenAIEmbeddings(model=Embedding_model))

In [5]:
tools = EunomiaTools(tool_names = ['read_doc', 'recheck_justification'], vectorstore=faiss_index).get_tools()
agent = Eunomia(tools=tools, model='gpt-4', get_cost=True)
agent.run(prompt="Read the document and find the MOFS and their water stability.\
            \Check your answers and if the reasoning is not valid, try to find better reasons.")



> Entering new AgentExecutor chain...
Action:
```
{
  "action": "read_doc",
  "action_input": {
    "type": "string"
  }
}
```This model's maximum context length is 8192 tokens. However, your messages resulted in 9134 tokens. Please reduce the length of the messages.

k=9 results hitting the token limit for the chosen LLM. Reducing k and retrying...

This model's maximum context length is 8192 tokens. However, your messages resulted in 8459 tokens. Please reduce the length of the messages.

k=8 results hitting the token limit for the chosen LLM. Reducing k and retrying...


Observation: 1. MOF: UiO-66
   Water Stability: Stable
   Probability Score: 0.9
   Justification: "UiO-66(Zr) is stable up to 450 8C under air and remains unaltered upon water adsorption/desorption cycles by switching reversibly between dehydroxylated and hydroxylated versions."

2. MOF: MIL-140A
   Water Stability: Stable
   Probability Score: 0.8
   Justification: "Thermogravimetric analysis (TGA) and IR spectr

ToolException: Too many arguments to single-input tool recheck_justification. Args: ['string', 'MIL-140A']